In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
# import torch.FloatTensor as Ft
import torch.nn as nn
from torch.autograd import Variable

plt.rcParams['font.family'] = 'FangSong'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def conv(x, y):
    return np.convolve(np.flip(x), y, 'valid')

In [3]:
def list_all_files(rootdir, key):
    _files = []
    list = os.listdir(rootdir)  # 列出文件夹下所有的目录与文件
    for i in range(0, len(list)):
        path = os.path.join(rootdir, list[i])
        if os.path.isdir(path):
            _files.extend(list_all_files(path, key))
        if os.path.isfile(path) and key in path:
            _files.append(path)
    return _files

In [4]:
def moving_avg(t, win):
    ret = np.zeros_like(t)
    for i in range(np.shape(t)[0]):
        if i == 0:
            ret[i] = t[i]
        elif i < win:
            # print(i, t[:i])
            ret[i] = np.mean(t[:i + 1])
        else:
            ret[i] = np.mean(t[i - win + 1:i + 1])
    return ret

In [5]:
def smooth(x, win):
    return np.apply_along_axis(
        lambda y: moving_avg(y, win),
        0, x
    )

In [7]:
def my_save_txt(path, tensor):
    with open(path, 'w') as outfile:
        outfile.write('# Array shape: {0}\n'.format(tensor.shape))
        for data_slice in tensor:
            np.savetxt(outfile, data_slice, fmt='%f')
            outfile.write('# New slice\n')

In [8]:
def load_data():
    # plt.switch_backend('QT5Agg')
    # figManager = plt.get_current_fig_manager()
    # figManager.window.showMaximized()
    # plt.ion()

    files = list_all_files('训练样本', '.txt')
    pos = []
    neg = []
    for f in files:
        # if '缓慢' in f:
        #     continue
        with open(f, 'r', encoding='utf-8') as fu:
            data = pd.read_csv(fu,
                               skiprows=1,
                               header=None,
                               delimiter=',').iloc[:max_len, :]
        start_col = 17 if 'left' in f else 19
        # rawdata = np.array(data.iloc[:, start_col:start_col + 2])
        forcesig = np.array(data.iloc[:, start_col + 8:start_col + 10])
        forcesig = smooth(forcesig, 10)
        if forcesig[0, 0] == 0 or forcesig[0, 1] == 0:
            print(f)
        forcesig = forcesig / forcesig[0]
        forcesig = np.pad(forcesig, ((0, max(0, max_len - np.shape(forcesig)[0])), (0, 0)), 'edge')
        if '缓慢' in f:
            pos.append(forcesig)
        else:
            neg.append(forcesig)

    pos, neg = np.array(pos), np.array(neg)
    my_save_txt('pos.dat', pos)
    my_save_txt('neg.dat', neg)
    # plt.plot(forcesig)
    # plt.hlines(0.3, 0, forcesig.shape[0], linestyles='--')
    # plt.title(f)
    # plt.savefig(''.join((
    #     'figs\\',
    #     'long press\\' if '缓慢' not in f else 'release\\',
    #     str(max_len),'\\',
    #     f[:-4].replace('\\', '-'), '.png'
    # )))
    # plt.pause(1)
    # plt.cla()
    return pos, neg



In [16]:
def easy_load_data():
    pos = np.loadtxt('pos.dat').reshape((781, 100, 2))
    neg = np.loadtxt('neg.dat').reshape((1655, 100, 2))
    return pos, neg

In [10]:
class DataLoader:
    def __init__(self, x, y, bs, gap, shuffle=False):
        self.m = x.shape[0]
        self.bs = bs
        self.shuffle = shuffle
        gap = int(self.m * gap)
        z = list(zip(x, y))
        np.random.shuffle(z)
        x_, y_ = zip(*z)
        self.train_size = gap
        self.x_train, self.x_test = x_[:gap], x_[gap:]
        self.y_train, self.y_test = y_[:gap], y_[gap:]

    def get_train_data(self):
        return self.x_train, self.y_train

    def get_test_data(self):
        return self.x_test, self.y_test

    def get_train_batch(self):
        xs, ys = self.x_train, self.y_train
        if self.shuffle:
            zs = list(zip(xs, ys))
            np.random.shuffle(zs)
            xs, ys = zip(*zs)
        if self.m <= self.bs:
            return xs, ys
        else:
            # print(self.train_size, self.bs, self.train_size // self.bs)
            for i in range(self.train_size // self.bs):
                yield xs[int(self.bs * i):int(self.bs * (i + 1))], \
                      ys[int(self.bs * i):int(self.bs * (i + 1))]
            if self.train_size % self.bs != 0:
                yield xs[int(self.bs * (self.train_size // self.bs)):], \
                      ys[int(self.bs * (self.train_size // self.bs)):]

In [11]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc1 = nn.Linear(max_len * hidden_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 64)

    def forward(self, x):
        # x   : 64 * 100 * 2
        # h, c: 2 * 100 * 32
        h0, c0 = torch.zeros(1, x.size(0), hidden_size), \
                 torch.zeros(1, x.size(0), hidden_size),
        out, (hn, cn) = self.rnn(x, (h0, c0))
        # print(out.size(), hn.size(), cn.size())
        # print(out[0,0,0],out[1,0,0])
        out = out.reshape(-1, 3200)
        # print(out[0,0],out[1,0])
        out = self.relu1(self.fc1(out))
        out = self.fc2(out)
        return out

In [14]:
max_len = 100
epochs = 1000
batch_size = 64
lr = 0.001
gap = 0.8
input_size = 2
hidden_size = 32
num_layers = 1
load_data()

训练样本\长按完整\17#（余振华）-45-不带保护套-left-0004.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0013.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0014.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0015.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0019.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0020.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0021.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0022.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0023.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0024.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0025.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0031.txt


C:\Users\wangye\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\wangye\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


训练样本\长按完整\17#（余振华）-45-不带保护套-left-0033.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0038.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0039.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0040.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0042.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-left-0051.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0001.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0008.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0009.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0012.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0020.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0023.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0035.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0038.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0040.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0041.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0042.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0043.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0044.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0047.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0048.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0049.txt
训练样本\长按完整\17#（余振华）-45-不带保护套-right-0051.txt
训练样本\长按完整\17#（余振华

训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0410.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0411.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0416.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0421.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0422.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0440.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0445.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0453.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0458.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0459.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0460.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-left-0463.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-right-0418.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-right-0420.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-right-0422.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-right-0423.txt
训练样本\长按完整\19#（魏阳光）-50-带保护套-right-0426.txt


(array([[[1.        , 1.        ],
         [0.98870056, 0.98529412],
         [0.96045198, 0.94117647],
         ...,
         [0.15254237, 0.11029412],
         [0.14689266, 0.11029412],
         [0.14689266, 0.11029412]],
 
        [[1.        , 1.        ],
         [1.        , 0.96666667],
         [0.99310345, 0.93333333],
         ...,
         [0.26206897, 0.07777778],
         [0.26206897, 0.07777778],
         [0.26206897, 0.07777778]],
 
        [[1.        , 1.        ],
         [1.        , 0.9868709 ],
         [0.98714653, 0.95404814],
         ...,
         [0.1285347 , 0.02625821],
         [0.1285347 , 0.02625821],
         [0.12596401, 0.02407002]],
 
        ...,
 
        [[1.        , 1.        ],
         [1.        , 1.        ],
         [0.98360656, 0.97916667],
         ...,
         [0.04918033, 0.09895833],
         [0.04371585, 0.09895833],
         [0.03825137, 0.09375   ]],
 
        [[1.        , 1.        ],
         [0.9844358 , 0.97938144],
       

In [ ]:
max_len = 100
epochs = 1000
batch_size = 64
lr = 0.001
gap = 0.8
input_size = 2
hidden_size = 32
num_layers = 1
P, N = easy_load_data()
print(P.shape, N.shape)
Y_p = np.ones((P.shape[0], 1))
Y_n = np.zeros((N.shape[0], 1))
x = np.concatenate((P, N), axis=0)
y = np.concatenate((Y_p, Y_n), axis=0)
loader = DataLoader(x, y, batch_size, gap, True)
print(x.shape, y.shape)
rnn = RNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr)
for ep in range(epochs):
    for (step, (bx, by)) in enumerate(loader.get_train_batch()):
        bx, by = np.array(bx), np.array(by)
        anchor = []
        positive = []
        negative = []
        for ib in range(bx.shape[0]):
            if by[ib] == 1:
                anchor.append(bx[ib])
                positive.append(bx[ib])
            else:
                negative.append(bx[ib])
        f_A = rnn(Variable(torch.FloatTensor(anchor)))
        f_P = rnn(Variable(torch.FloatTensor(positive)))
        f_N = rnn(Variable(torch.FloatTensor(negative)))
        dAP = Variable(torch.FloatTensor([
            torch.max(
                torch.norm(f_A[i] - f_P, dim=1)
            ) for i in range(f_A.size(0))
        ]).sum(), requires_grad=True)
        dAN = Variable(torch.FloatTensor([
            torch.min(
                torch.norm(f_A[i] - f_N, dim=1)
            ) for i in range(f_A.size(0))
        ]).sum(), requires_grad=True)
        print(dAP.item(),dAN.item())
        loss = dAP - dAN + 0.2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(ep, step, bx.shape, by.shape, np.shape(anchor), loss.item())

(781, 100, 2) (1655, 100, 2)
(2436, 100, 2) (2436, 1)
0.5878128409385681 nan
0 0 (64, 100, 2) (64, 1) (17, 100, 2) nan
1.1031144857406616 nan
0 1 (64, 100, 2) (64, 1) (20, 100, 2) nan
1.0781720876693726 nan
0 2 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.559542715549469 nan
0 3 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.746056079864502 nan
0 4 (64, 100, 2) (64, 1) (17, 100, 2) nan
1.5409882068634033 nan
0 5 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.8713479042053223 nan
0 6 (64, 100, 2) (64, 1) (23, 100, 2) nan
1.1697858572006226 nan
0 7 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.6895129680633545 nan
0 8 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.1266694068908691 nan
0 9 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.3637040853500366 nan
0 10 (64, 100, 2) (64, 1) (24, 100, 2) nan
1.2310028076171875 nan
0 11 (64, 100, 2) (64, 1) (28, 100, 2) nan
0.9448727369308472 nan
0 12 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.7063376307487488 nan
0 13 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.6600059866905212 nan
0 14 (64, 1

0.8752901554107666 nan
4 1 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.8394647836685181 nan
4 2 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.4629944562911987 nan
4 3 (64, 100, 2) (64, 1) (28, 100, 2) nan
1.4270482063293457 nan
4 4 (64, 100, 2) (64, 1) (26, 100, 2) nan
1.195418357849121 nan
4 5 (64, 100, 2) (64, 1) (25, 100, 2) nan
1.4014519453048706 nan
4 6 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.8022379875183105 nan
4 7 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.043096661567688 nan
4 8 (64, 100, 2) (64, 1) (25, 100, 2) nan
1.0561989545822144 nan
4 9 (64, 100, 2) (64, 1) (26, 100, 2) nan
0.6261066794395447 nan
4 10 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.6756447553634644 nan
4 11 (64, 100, 2) (64, 1) (13, 100, 2) nan
1.0578490495681763 nan
4 12 (64, 100, 2) (64, 1) (25, 100, 2) nan
0.7534902095794678 nan
4 13 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.7250734567642212 nan
4 14 (64, 100, 2) (64, 1) (20, 100, 2) nan
0.7818959355354309 nan
4 15 (64, 100, 2) (64, 1) (20, 100, 2) nan
1.171783685684204 nan

8 2 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.2945051193237305 nan
8 3 (64, 100, 2) (64, 1) (22, 100, 2) nan
1.1797854900360107 nan
8 4 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.7948505878448486 nan
8 5 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.3926301002502441 nan
8 6 (64, 100, 2) (64, 1) (25, 100, 2) nan
1.4630786180496216 nan
8 7 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.7891567349433899 nan
8 8 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.8588834404945374 nan
8 9 (64, 100, 2) (64, 1) (15, 100, 2) nan
0.7210449576377869 nan
8 10 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.7029222846031189 nan
8 11 (64, 100, 2) (64, 1) (19, 100, 2) nan
1.2624009847640991 nan
8 12 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.6001626253128052 nan
8 13 (64, 100, 2) (64, 1) (17, 100, 2) nan
1.0999279022216797 nan
8 14 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.2755175828933716 nan
8 15 (64, 100, 2) (64, 1) (28, 100, 2) nan
0.9643445611000061 nan
8 16 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.5985578298568726 nan
8 17 (64, 100, 2) 

12 2 (64, 100, 2) (64, 1) (23, 100, 2) nan
1.0998198986053467 nan
12 3 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.6921256184577942 nan
12 4 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.7724100351333618 nan
12 5 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.8486717939376831 nan
12 6 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.6830261945724487 nan
12 7 (64, 100, 2) (64, 1) (20, 100, 2) nan
0.912042498588562 nan
12 8 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.5347538590431213 nan
12 9 (64, 100, 2) (64, 1) (15, 100, 2) nan
0.42031896114349365 nan
12 10 (64, 100, 2) (64, 1) (15, 100, 2) nan
0.7870622277259827 nan
12 11 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.5888045430183411 nan
12 12 (64, 100, 2) (64, 1) (15, 100, 2) nan
0.8970487117767334 nan
12 13 (64, 100, 2) (64, 1) (23, 100, 2) nan
1.612721562385559 nan
12 14 (64, 100, 2) (64, 1) (32, 100, 2) nan
0.49153846502304077 nan
12 15 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.7731925845146179 nan
12 16 (64, 100, 2) (64, 1) (19, 100, 2) nan
1.2871415615081787 nan
12 

1.119489073753357 nan
16 3 (64, 100, 2) (64, 1) (24, 100, 2) nan
1.5606112480163574 nan
16 4 (64, 100, 2) (64, 1) (30, 100, 2) nan
0.6072371006011963 nan
16 5 (64, 100, 2) (64, 1) (15, 100, 2) nan
0.7954020500183105 nan
16 6 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.7993998527526855 nan
16 7 (64, 100, 2) (64, 1) (20, 100, 2) nan
1.041126012802124 nan
16 8 (64, 100, 2) (64, 1) (22, 100, 2) nan
1.1484532356262207 nan
16 9 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.9465018510818481 nan
16 10 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.4696389436721802 nan
16 11 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.602205216884613 nan
16 12 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.8788358569145203 nan
16 13 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.6126551032066345 nan
16 14 (64, 100, 2) (64, 1) (13, 100, 2) nan
1.0728930234909058 nan
16 15 (64, 100, 2) (64, 1) (26, 100, 2) nan
1.0912508964538574 nan
16 16 (64, 100, 2) (64, 1) (24, 100, 2) nan
1.7056012153625488 nan
16 17 (64, 100, 2) (64, 1) (29, 100, 2) nan
0.993

...

0.6407579779624939 nan
184 0 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.8265220522880554 nan
184 1 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.159356951713562 nan
184 2 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.8574767708778381 nan
184 3 (64, 100, 2) (64, 1) (22, 100, 2) nan
1.1227155923843384 nan
184 4 (64, 100, 2) (64, 1) (26, 100, 2) nan
0.7066065073013306 nan
184 5 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.2262378931045532 nan
184 6 (64, 100, 2) (64, 1) (30, 100, 2) nan
0.6169522404670715 nan
184 7 (64, 100, 2) (64, 1) (15, 100, 2) nan
1.1158294677734375 nan
184 8 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.993607759475708 nan
184 9 (64, 100, 2) (64, 1) (23, 100, 2) nan
0.9368029236793518 nan
184 10 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.9410291314125061 nan
184 11 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.9951004385948181 nan
184 12 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.8211476802825928 nan
184 13 (64, 100, 2) (64, 1) (24, 100, 2) nan
1.1676684617996216 nan
184 14 (64, 100, 2) (64, 1) (25, 100,

187 30 (28, 100, 2) (28, 1) (9, 100, 2) 0.1527486890554428
0.5584114789962769 nan
188 0 (64, 100, 2) (64, 1) (14, 100, 2) nan
1.0855094194412231 nan
188 1 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.853880763053894 nan
188 2 (64, 100, 2) (64, 1) (23, 100, 2) nan
1.1615358591079712 nan
188 3 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.7997188568115234 nan
188 4 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.0631262063980103 nan
188 5 (64, 100, 2) (64, 1) (24, 100, 2) nan
1.253900170326233 nan
188 6 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.7055532932281494 nan
188 7 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.8336946368217468 nan
188 8 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.7967202663421631 nan
188 9 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.6885687112808228 nan
188 10 (64, 100, 2) (64, 1) (15, 100, 2) nan
1.1906377077102661 nan
188 11 (64, 100, 2) (64, 1) (23, 100, 2) nan
0.8370987772941589 nan
188 12 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.8848391771316528 nan
188 13 (64, 100, 2) (64, 1) (19, 100, 2) nan
0

0.6637169122695923 nan
191 30 (28, 100, 2) (28, 1) (12, 100, 2) nan
0.8912367224693298 nan
192 0 (64, 100, 2) (64, 1) (23, 100, 2) nan
0.8294973969459534 nan
192 1 (64, 100, 2) (64, 1) (23, 100, 2) nan
0.7767910361289978 nan
192 2 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.9962103366851807 nan
192 3 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.9476916790008545 nan
192 4 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.1033532619476318 nan
192 5 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.63749760389328 nan
192 6 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.3934727907180786 nan
192 7 (64, 100, 2) (64, 1) (26, 100, 2) nan
1.0413464307785034 nan
192 8 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.8538788557052612 nan
192 9 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.5967536568641663 nan
192 10 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.243177890777588 nan
192 11 (64, 100, 2) (64, 1) (26, 100, 2) nan
1.0933525562286377 nan
192 12 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.6307446956634521 nan
192 13 (64, 100, 2) (64, 1) (17, 100, 

195 30 (28, 100, 2) (28, 1) (13, 100, 2) nan
1.2323050498962402 nan
196 0 (64, 100, 2) (64, 1) (26, 100, 2) nan
0.9993070363998413 nan
196 1 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.687674880027771 nan
196 2 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.9122103452682495 nan
196 3 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.5639448761940002 nan
196 4 (64, 100, 2) (64, 1) (21, 100, 2) nan
0.8462048768997192 nan
196 5 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.381464958190918 nan
196 6 (64, 100, 2) (64, 1) (28, 100, 2) nan
0.792126476764679 nan
196 7 (64, 100, 2) (64, 1) (20, 100, 2) nan
0.571894645690918 nan
196 8 (64, 100, 2) (64, 1) (18, 100, 2) nan
1.0903764963150024 nan
196 9 (64, 100, 2) (64, 1) (20, 100, 2) nan
0.6526352763175964 nan
196 10 (64, 100, 2) (64, 1) (17, 100, 2) nan
0.8040308952331543 nan
196 11 (64, 100, 2) (64, 1) (27, 100, 2) nan
0.9200193285942078 nan
196 12 (64, 100, 2) (64, 1) (21, 100, 2) nan
1.1071898937225342 nan
196 13 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.785277783870697

0.6141621470451355 0.14347383379936218
199 30 (28, 100, 2) (28, 1) (11, 100, 2) 0.6706883311271667
0.7378974556922913 nan
200 0 (64, 100, 2) (64, 1) (29, 100, 2) nan
0.887166440486908 nan
200 1 (64, 100, 2) (64, 1) (19, 100, 2) nan
0.7672602534294128 nan
200 2 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.8960772156715393 nan
200 3 (64, 100, 2) (64, 1) (24, 100, 2) nan
0.875688374042511 nan
200 4 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.6784274578094482 nan
200 5 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.853795051574707 nan
200 6 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.9732542037963867 nan
200 7 (64, 100, 2) (64, 1) (23, 100, 2) nan
0.7997716069221497 nan
200 8 (64, 100, 2) (64, 1) (22, 100, 2) nan
0.825026273727417 nan
200 9 (64, 100, 2) (64, 1) (20, 100, 2) nan
1.1557402610778809 nan
200 10 (64, 100, 2) (64, 1) (26, 100, 2) nan
1.0585507154464722 nan
200 11 (64, 100, 2) (64, 1) (18, 100, 2) nan
0.8485286235809326 nan
200 12 (64, 100, 2) (64, 1) (20, 100, 2) nan
0.6730613708496094 nan
200 13 (